<a href="https://colab.research.google.com/github/julianavelasquezg/Business-Intelligence/blob/main/Reto_1_Modelo_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reto 1 - Modelo de clasificación (Naive Bayes)**

De acuerdo con la base de datos Longitudinal Survey (Apendice A, Conjunto 5: Datos de Levantamiento Longitudinal del Libro Guía Analítica de Negocios Comunicación con Datos), llevar a cabo la implementación de un modelo Naive Bayes para determinar si una persona vive en un área urbana o en un área rural.

Para este modelo, tomar las variables socioeconómicas como: **Edad, Educación de los Padres, Estatura, Peso, Ingresos, Escala de Autoestima, Tamaño de la Familia.**
Para el análisis de los resultados, es necesario describir cada una de las variables que conforman la base de datos, eliminando de plano los registros que poseen datos faltantes.
**Preguntas (Longitudinal Survey):**
- ¿Cuál es el número de datos total de la base de datos?
- ¿Cuál es el porcentaje de los datos que hacen parte de cada una de las categorías de clasificación?
- Determinar el intervalo en el cual se encuentra la variación de la Edad
- Determinar el intervalo en el cual se encuentra la variación de la Altura
- Determinar el intervalo en el cual se encuentra la variación del Peso de las Personas
- ¿Cuál es la exactitud alcanzada por el modelo?
- ¿Cuál es la tasa de error alcanzada por el modelo?
- ¿Cuál es la Sensibilidad alcanzada por el modelo?
- ¿Cuál es la Especificidad alcanzada por el modelo?
- ¿Para una persona con las siguientes características ['Age','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income']=[34,12,15,2,24,69,160,48000] indicar si vive en la ciudad o las áreas rurales?



**0. Cargar librerías de trabajo**

In [ ]:
import numpy as np #Posee funciones matemáticas
import pandas as pd #Comunicación con el excel

**1. Cargar la base de datos**

In [ ]:
nxl='/content/1. BD1_Longitudinal Survey_Int.xlsx'
XDB=pd.read_excel(nxl)



**2. Seleccionar las variables con las que trabajaremos: 'Age','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income'**

In [ ]:
XDB=XDB[['Age','Urban','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income']]
XDB.head(10)

,Age,Urban,Mother_Edu,Father_Edu,FamilySize,Self_Esteem,Height,Weight,Income
0,21,1.0,8.0,8.0,5,NaN,65.0,NaN,NaN
1,20,1.0,5.0,8.0,5,16.0,62.0,120.0,0.0
2,18,1.0,10.0,12.0,5,20.0,NaN,NaN,0.0
3,17,1.0,11.0,12.0,5,NaN,67.0,110.0,NaN
4,20,1.0,12.0,12.0,4,23.0,63.0,130.0,NaN
5,19,1.0,12.0,12.0,4,27.0,64.0,200.0,40000.0
6,15,1.0,12.0,12.0,3,26.0,65.0,131.0,25000.0
7,21,1.0,9.0,6.0,3,23.0,65.0,179.0,27400.0
8,16,1.0,12.0,10.0,6,26.0,66.0,145.0,52000.0
9,19,1.0,12.0,12.0,3,19.0,66.0,115.0,NaN


Porcentaje de respuestas por cada variable

In [ ]:
for columna in XDB.columns:
  num_respuestas = XDB[columna].count()
  total_respuestas = len(XDB)
  porcentaje = (num_respuestas / total_respuestas) * 100
  print(f"Porcentaje de respuestas en la variable '{columna}': {porcentaje:.2f}%")

Porcentaje de respuestas en la variable 'Age': 100.00%
Porcentaje de respuestas en la variable 'Urban': 99.68%
Porcentaje de respuestas en la variable 'Mother_Edu': 93.63%
Porcentaje de respuestas en la variable 'Father_Edu': 85.76%
Porcentaje de respuestas en la variable 'FamilySize': 100.00%
Porcentaje de respuestas en la variable 'Self_Esteem': 94.53%
Porcentaje de respuestas en la variable 'Height': 95.72%
Porcentaje de respuestas en la variable 'Weight': 95.63%
Porcentaje de respuestas en la variable 'Income': 59.70%


**3. Contamos cuantas personas están en la base de datos**

In [ ]:
total_respondents = len(XDB)

print(f"Número total de personas que respondieron: {total_respondents}")

Número total de personas que respondieron: 12686


**4. Eliminamos las personas que tengan datos faltantes**

In [ ]:
# Eliminar filas con al menos un valor faltante
XDB = XDB.dropna()
XDB.head(10)

,Age,Urban,Mother_Edu,Father_Edu,FamilySize,Self_Esteem,Height,Weight,Income
1,20,1.0,5.0,8.0,5,16.0,62.0,120.0,0.0
5,19,1.0,12.0,12.0,4,27.0,64.0,200.0,40000.0
6,15,1.0,12.0,12.0,3,26.0,65.0,131.0,25000.0
7,21,1.0,9.0,6.0,3,23.0,65.0,179.0,27400.0
8,16,1.0,12.0,10.0,6,26.0,66.0,145.0,52000.0
12,21,1.0,12.0,16.0,5,25.0,71.0,180.0,60000.0
13,16,1.0,12.0,12.0,5,21.0,67.0,135.0,48000.0
14,15,1.0,12.0,12.0,4,23.0,73.0,185.0,0.0
15,21,1.0,12.0,12.0,4,25.0,63.0,130.0,38000.0
16,22,1.0,12.0,15.0,2,24.0,69.0,160.0,48000.0


**4. Contamos ahora cuántas personas quedan con datos para evaluar el modelo después de eliminar los que tenían datos faltantes**

In [ ]:
total_respondents = len(XDB)

print(f"Número total de personas que respondieron: {total_respondents}")

Número total de personas que respondieron: 5948


**Ahora verificamos qué porcentaje de las personas de la base de datos viven en zonas urbanas o rurales**

In [ ]:
# Porcentaje de cada categoría en "Urban"
categories = XDB['Urban'].value_counts(normalize=True) * 100
print('Porcentaje de datos en cada categoría:\n', categories)
#(Urbano) 1.0 → 79.16%: Significa que el 79.16% de las personas en la base de datos viven en áreas urbanas.
#(Rural) 0.0 → 20.83%: Indica que el 20.83% de las personas viven en áreas rurales.

Porcentaje de datos en cada categoría:
 Urban
1.0    79.169469
0.0    20.830531
Name: proportion, dtype: float64


**4. Separar las variables de entrada y salida. En este caso la de salida sería 'Urban', y de entrada serían 'Age','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income'**

In [ ]:
XD=XDB[['Age','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income']] #XD -> Variables de entrada
YD=XDB[['Urban']] #YD -> Variables de salida (Variable pronóstico)


- Determinar el intervalo en el cual se encuentra la variación de la Edad
- Determinar el intervalo en el cual se encuentra la variación de la Altura
- Determinar el intervalo en el cual se encuentra la variación del Peso de las Personas


In [ ]:
# Intervalo de la Edad
age_min = XDB['Age'].min()
age_max = XDB['Age'].max()
age_interval = (age_min, age_max)

# Intervalo de la Altura
height_min = XDB['Height'].min()
height_max = XDB['Height'].max()
height_interval = (height_min, height_max)

# Intervalo del Peso
weight_min = XDB['Weight'].min()
weight_max = XDB['Weight'].max()
weight_interval = (weight_min, weight_max)

# Imprimir los resultados
print(f"Intervalo de la Edad: {age_interval}")
print(f"Intervalo de la Altura: {height_interval}")
print(f"Intervalo del Peso: {weight_interval}")

Intervalo de la Edad: (15, 22)
Intervalo de la Altura: (51.0, 80.0)
Intervalo del Peso: (72.0, 375.0)


**5. Implementación del modelo**

In [ ]:
from sklearn.naive_bayes import GaussianNB #Se importa la librería de Naive Bayes

mnb=GaussianNB() # Aloja el modelo
mnb.fit(XD,YD) #Entrena el modelo

#Comenzamos obteniendo la información del modelo
u=mnb.theta_ #La media de las variables analizadas es
print("La media de las variables analizadas es:\n",u)

var=mnb.var_ # Se calculan las varianzas (qué tan alejados estan los datos de la media)
print("La varianza de las variables analizadas es:\n",var)
sigma=np.sqrt(var) #Se calcula la desviación estándar
print("La desviación estándar de las variables analizadas es:\n",sigma)

#Se determinan los limites superiores e inferiores de las variables analizadas
LS=u+sigma
LI=u-sigma
print("El límite superior de las variables analizadas es:\n",LS)
print("El límite inferior de las variables analizadas es:\n",LI)

La media de las variables analizadas es:
 [[1.83640032e+01 1.07312349e+01 1.02711864e+01 4.73930589e+00
  2.19951574e+01 6.72429379e+01 1.49003228e+02 3.01248854e+04]
 [1.83618603e+01 1.11613931e+01 1.12331705e+01 4.68103631e+00
  2.25744319e+01 6.69116585e+01 1.44750903e+02 3.32736742e+04]]
La varianza de las variables analizadas es:
 [[6.11377773e+00 9.96419474e+00 1.51194652e+01 6.51406931e+00
  1.77789403e+01 1.66650620e+01 9.21039647e+02 1.00161093e+09]
 [6.11792319e+00 1.15645045e+01 1.69373304e+01 5.96364977e+00
  1.76201044e+01 1.76562243e+01 9.01712618e+02 1.30246862e+09]]
La desviación estándar de las variables analizadas es:
 [[2.47260545e+00 3.15661127e+00 3.88837565e+00 2.55226748e+00
  4.21650807e+00 4.08228636e+00 3.03486350e+01 3.16482374e+04]
 [2.47344359e+00 3.40066236e+00 4.11549881e+00 2.44205851e+00
  4.19763081e+00 4.20193102e+00 3.00285301e+01 3.60897302e+04]]
El límite superior de las variables analizadas es:
 [[2.08366087e+01 1.38878461e+01 1.41595621e+01 7.291

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


¿Para una persona con las siguientes características ['Age','Mother_Edu','Father_Edu','FamilySize','Self_Esteem','Height','Weight','Income']=[34,12,15,2,24,69,160,48000] indicar si vive en la ciudad o las áreas rurales?

In [ ]:
XDS=[[34,12,15,2,24,69,160,48000]] #Como un valor de entrada
YDP=mnb.predict(XDS) #Se obtiene la predicción
print("La predicción del individuo es:\n",YDP)


La predicción del individuo es:
 [1.]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


**Exactitud del modelo**

In [ ]:
from sklearn.metrics import confusion_matrix

#Haga una revisión de los datos
YDP2=mnb.predict(XD) #Haga el pronostico para los datos base

cm=confusion_matrix(YD,YDP2) #Cree la matriz de confusión
print("La matriz de confusión es:\n",cm)

VN=cm[0,0];FP=cm[0,1];VP=cm[1,1];FN=cm[1,0]

#Métricas desempeño
Ex=(VP+VN)/(VP+VN+FP+FN) #Exactitud: comportamiento general urbano
print("La exactitud del modelo es:\n",Ex)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error: comportamiento general rural
print("La tasa de error del modelo es:\n",TE)

Se=VP/(VP+FN) #Sensibilidad: comportamiento correcto urbano
print("La sensibilidad del modelo es:\n",Se)

Sp=VN/(VN+FP) #Especificidad: comportamiento correcto rural
              #Sobre el total de los rurales
print("La especificidad del modelo es:\n",Sp)



La matriz de confusión es:
 [[   3 1236]
 [   8 4701]]
La exactitud del modelo es:
 0.7908540685944856
La tasa de error del modelo es:
 0.20914593140551446
La sensibilidad del modelo es:
 0.9983011255043533
La especificidad del modelo es:
 0.002421307506053269


**Análisis de resultados**

De acuerdo con los resultados obtenidos del modelo Naive Bayes  para la base de datos proporcionada, se utilizaron las variables 'Age', 'Mother_Edu', 'Father_Edu', 'FamilySize', 'Self_Esteem', 'Height', 'Weight', y 'Income' para determinar si las personas de la base de datos viven en un lugar 'Urbano' o 'Rural'. La base de datos contiene un total de 12686 registros, de los cuales después de eliminar a las personas que tenían datos incompletos obtuvimos un total de 5984 datos viables para el análisis, con un (79.16%) de datos correspondientes a las personas que viven en un área urbana y un (20.83%) a las personas que viven en un área rural. El modelo alcanzó una exactitud del (79.09%), lo que implica que el casi el 80% de las predicciones de la clasificación de residencia para el área urbana fueron correctas. La tasa de error es del (20.91%), lo que significa que el modelo cometió un 20.91% de errores en sus predicciones de las personas que viven en un área rural. En cuanto a la sensibilidad, el modelo mostró un valor muy positivo de (99.83%), lo que indica que tiene una capacidad sobresaliente para identificar de manera correcta los casos de las personas que viven en sectores urbanos. Sin embargo, la especificidad es muy baja (0.24%), lo que refleja la dificultad del modelo para predecir correctamente los casos de las personas que habitan en lugares rurales. La precisión del modelo alcanzó un (79.18%), lo que significa que de manera correcta puede predecir las personas que viven en áreas urbanas. Finalmente, la precisión negativa fue baja (27.27%), lo que destaca una deficiencia en la predicción de las personas que viven en un sector rural. Podemos concluir entonces que el modelo es muy útil para identificar los casos de las personas que viven en áreas urbanas, sin embargo tiende a ser menos preciso en los casos de residencias rurales.